In [ ]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt
import re

# import logging
# logging.basicConfig(level=logging.DEBUG)

import altair as alt
alt.data_transformers.disable_max_rows()

from src.data.query_snowflake import update_week_extremes, read_DALI_week_extremes, read_meta
from src.plot.altair import plot_limits, plot_history
from src.plot.formatting import format_limits, format_history

In [ ]:
# update_week_extremes()

In [ ]:
sample_box = "ESD.000400-1"

In [ ]:
%%time 
df_data = read_DALI_week_extremes(boxid=sample_box, L="sumli")
df_data

In [ ]:
%%time
df_meta = read_meta(sample_box)
df_meta

In [ ]:
df_plot = format_history(df_data)

In [ ]:
df_limits = format_limits(df_meta=df_meta)

In [ ]:
plot_limits(df_limits) + plot_history(df_plot)